# Covid-19 Dashboard - Test to Case Comparisons

Public health England publish daily status' showing the number of new tests conducted under the Covid-19 Pandemic and the results. 

This dashboard allows you to visualise this data in a way that you can determine the relationship between tests conducted and the number of cases reported as a result. 

This is with a view to build a perspective on the US claim of more cases only being discovered due to increased testing. 

You can refresh this data on a daily basis as and when Public Health England make it available by making use of the provided 'Refresh Button'

In [240]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [241]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [242]:
filters = [
    'areaType=overview' #data for the entire United Kingdom
]
structure = {
    "date": "date",
    "casesPub": "newCasesByPublishDate",
    "casesSpec": "newCasesBySpecimenDate",
    "testsNew": "newTestsByPublishDate",
    "testsTotalPub": "cumTestsByPublishDate", 
    "testsTotalSpec": "cumCasesBySpecimenDate"
    '''this data will allow for further analysis at a later date to compare total numbers of tests and cases'''
}
api = Cov19API(filters=filters, structure=structure) #define the api
testcasedata=api.get_json() 
with open("testcasedata.json", "wt") as OUTF:
    json.dump(testcasedata, OUTF)

In [243]:
testcasedatalist=testcasedata['data'] #create a list from the api, original was a dictionary 

testdates=[dictionary['date'] for dictionary in testcasedatalist ]
testdates.sort()# sort the dates in chronological order

In [244]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [276]:
startdate=parse_date(testdates[0])
enddate=parse_date(testdates[-1])
print (startdate, ' to ', enddate)

2020-01-03 00:00:00  to  2020-11-26 00:00:00


In [277]:
index=pd.date_range(startdate, enddate, freq='D')
newcasetest=pd.DataFrame(index=index, columns=['casesPub', 'casesSpec', 'testsNew']) #create the data frame
newcasetest

,casesPub,casesSpec,testsNew
2020-01-03,NaN,NaN,NaN
2020-01-04,NaN,NaN,NaN
2020-01-05,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN
...,...,...,...
2020-11-22,NaN,NaN,NaN
2020-11-23,NaN,NaN,NaN
2020-11-24,NaN,NaN,NaN
2020-11-25,NaN,NaN,NaN


In [278]:
for entry in testcasedatalist: # populate the data frame with data from the list created
    date=parse_date(entry['date'])
    for column in ['casesPub', 'casesSpec', 'testsNew']:
        if pd.isna(newcasetest.loc[date, column]):  
            value= float(entry[column]) if entry[column]!=None else 0.0 # display null data with 0 instead of none
            newcasetest.loc[date, column]=value
            
newcasetest.fillna(0.0, inplace=True)
            
newcasetest

,casesPub,casesSpec,testsNew
2020-01-03,0.0,0.0,0.0
2020-01-04,0.0,0.0,0.0
2020-01-05,0.0,0.0,0.0
2020-01-06,0.0,0.0,0.0
2020-01-07,0.0,0.0,0.0
...,...,...,...
2020-11-22,18662.0,11729.0,279458.0
2020-11-23,15450.0,17798.0,214920.0
2020-11-24,11299.0,11242.0,314800.0
2020-11-25,18213.0,0.0,359597.0


## Tests Against Cases

The US President had publicly stated his hypothesis that the reason that more Covid cases were arising were due to increase testing. 

Through the gathering of UK health data and the plotting of this against the metrics of cases and tests, a visual representation of the relationship between increased testing and cases found can be seen. 

From the data to date (which can be refreshed below), the US Presidents hypothesis is proven incorrect. This is very clear between May and October 2020 where testing increased by 500%, and cases in most case levelled or decreased. 

#### Key: 

  - casesPub: The number of cases recored by publish date 
  - casesSpec: The number of cases recorded by Specimen date 
  - testsNew: The number of new tests recorded

By default, for your interest all metrics are plotted. 

An interactive control is available which can be used to add or remove metrics. 

In [279]:
def access_api(api):
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    return {} # return data read from the API

In [288]:
casetestcols=wdg.SelectMultiple(
    options=['casesPub', 'casesSpec', 'testsNew'],
    value=['casesPub', 'casesSpec', 'testsNew'],
    rows=3,
    description='Stats',
    disabled=False
)

def casetestgraph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        newcasetest.plot(kind='line', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in variable output    
casetestgraph=wdg.interactive_output(casetestgraph, {'graphcolumns': casetestcols})

display(casetestcols, casetestgraph)

SelectMultiple(description='Stats', index=(0, 1, 2), options=('casesPub', 'casesSpec', 'testsNew'), rows=3, va…

Output()

In [274]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. 
    apidata=Cov19API(filters=filters, structure=structure)
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=access_api(newcasetest)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Fetch Data', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="click button to refresh data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='refresh'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='success', description='Fetch Data', icon='refresh', style=ButtonStyle(), tooltip='click b…

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*